In [7]:
from ultralytics import YOLO
import time
import torch
import pandas as pd
import openpyxl
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np
import tkinter as tk
from tkinter import Label, Button, Frame, Text, Scrollbar, VERTICAL, RIGHT, Y
from PIL import Image, ImageTk
from openpyxl.drawing.image import Image as ExcelImage
import tempfile
import os
from tkinter import filedialog, Tk
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment,Border,Side



In [8]:
%pip install --upgrade openpyxl


Note: you may need to restart the kernel to use updated packages.


In [9]:
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker
deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=80)

In [ ]:
'''# Initialize the YOLO model
model = YOLO("yolov8m.pt")  # Load the model once before the loop

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera, change if needed

# Initialize variables
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()
unique_track_ids = set()
track_labels = {}
track_times = {}
track_positions = {}
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the video

# Speed threshold to determine if a person is running
running_threshold = 0.5  # Adjust this value based on trial and error

# Variables for obstruction detection
obstruction_threshold = 0.60  # Percentage of the frame that is considered blocked
obstruction_frames = 0  # Counter for blocked frames
obstruction_blocked = False  # Whether the camera is blocked

# List to store IDs of persons who have stayed more than 1 minute
alert_person_ids = []

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        # Flip the frame horizontally to correct the mirrored effect
        frame = cv2.flip(frame, 1)

        og_frame = frame
        frame = og_frame.copy()

        results = model(frame, device=0, classes=0, conf=0.8)

        class_names = ['person']

        active_track_ids = set()

        for result in results:
            boxes = result.boxes
            cls = boxes.cls.tolist()
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh

            for class_index in cls:
                class_name = class_names[int(class_index)]
                print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh.cpu().numpy()

        tracks = tracker.update(bboxes_xywh, conf, og_frame)

        for track in tracker.tracker.tracks:
            track_id = track.track_id
            hits = track.hits
            x1, y1, x2, y2 = track.to_tlbr()
            w = x2 - x1
            h = y2 - y1

            # Define color for bounding box
            red_color = (0, 0, 255)
            blue_color = (255, 0, 0)
            green_color = (0, 255, 0)

            color_id = track_id % 3
            color = red_color if color_id == 0 else blue_color if color_id == 1 else green_color

            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

            # Track the label and start time for the detected object
            if track_id not in track_labels:
                track_labels[track_id] = "Person"  # Initialize label as "Person"
                track_times[track_id] = 0  # Initialize the time spent
                track_positions[track_id] = (x1, y1)  # Initialize the starting position

            # Update the time spent (in frames)
            track_times[track_id] += 1

            # Calculate the displacement (speed) between frames
            prev_x1, prev_y1 = track_positions[track_id]
            displacement = np.sqrt((x1 - prev_x1) ** 2 + (y1 - prev_y1) ** 2)
            speed = displacement / fps  # Speed in pixels per frame

            # Update the position
            track_positions[track_id] = (x1, y1)

            # Check if the person is running based on speed and bounding box size (to filter out non-person movements)
            if speed > running_threshold and w * h > 5000:  # Adjust size threshold as needed
                track_labels[track_id] = "Running"  # Update label to 'Running'
            else:
                track_labels[track_id] = "Person"  # Otherwise, keep label as 'Person'

            # Calculate time spent in minutes and seconds
            total_seconds = track_times[track_id] / fps
            minutes = int(total_seconds // 60)
            seconds = int(total_seconds % 60)

            # Check if the person has stayed more than 1 minute
            if total_seconds > 60 and track_id not in alert_person_ids:
                alert_person_ids.append(track_id)

            text_color = (0, 0, 0)
            cv2.putText(og_frame, f"{track_labels[track_id]} {minutes:02}:{seconds:02}", 
                        (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

            active_track_ids.add(track_id)

        lost_track_ids = unique_track_ids - active_track_ids
        for lost_track_id in lost_track_ids:
            track_labels.pop(lost_track_id, None)
            track_times.pop(lost_track_id, None)
            track_positions.pop(lost_track_id, None)

        unique_track_ids.intersection_update(active_track_ids)
        unique_track_ids.update(active_track_ids)

        person_count = len(unique_track_ids)

        # Check if the frame is obstructed
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.Laplacian(gray_frame, cv2.CV_64F).var()  # Use variance of Laplacian to detect blur (obstruction)
        if blur < 50:  # Threshold to detect blurriness or obstruction
            obstruction_frames += 1
            if obstruction_frames > fps * 2:  # If obstructed for more than 2 seconds
                obstruction_blocked = True
        else:
            obstruction_frames = 0
            obstruction_blocked = False

        current_time = time.perf_counter()
        elapsed = (current_time - start_time)
        counter += 1
        if elapsed > 1:
            fps = counter / elapsed
            counter = 0
            start_time = current_time

        cv2.putText(og_frame, f"Person Count: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display the obstruction notification
        if obstruction_blocked:
            cv2.putText(og_frame, "Camera View Blocked", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Display alert for persons who have stayed more than 1 minute
        if alert_person_ids:
            cv2.putText(og_frame, f"Alert! Person IDs: {', '.join(map(str, alert_person_ids))} over 1 minute", 
                        (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Video", 960, 540)
        cv2.imshow("Video", og_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()'''


In [ ]:
#SampleCommit - Prolong Stay

In [9]:
def save_screenshot(frame, csv_path, row_num):
    """Function to save the screenshot to the Excel file."""
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp_file:
        screenshot_path = tmp_file.name
        cv2.imwrite(screenshot_path, frame)

    if csv_path:
        workbook = openpyxl.load_workbook(csv_path)
        sheet = workbook.active
        img = ExcelImage(screenshot_path)
        img.anchor = f'F{row_num}'  # Place image in column F at the correct row
        sheet.add_image(img)
        workbook.save(csv_path)
        os.remove(screenshot_path)

In [ ]:
# Initialize YOLO model and DeepSort tracker
model = YOLO('yolov10m.pt')


# Tkinter window setup
window = tk.Tk()
window.title("Real-time Object Detection")
window.geometry("1920x1080")

# Global variables
cap = None
running = False  # To control the webcam feed loop
panel = None  # Panel for video display
info_text = None  # Text widget to display inference information

# Static variables to track changes
current_person_count = 0
current_camera_view_status = "Normal"
current_running_status = "No Running Detected"
alert_person_ids = []

# Variables for tracking
counter, fps, elapsed = 0, 1, 0  # Initialize fps to 1 to avoid division by zero
start_time = time.perf_counter()
unique_track_ids = set()
track_labels = {}
track_times = {}
track_positions = {}
obstruction_frames = 0
obstruction_blocked = False
running_threshold = 0.5
obstruction_threshold = 0.60
running_counters = {}  # Dictionary to track consecutive frames of running detection


def start_detection():
    """Function to start the object detection process."""
    global cap, running, fps
    if not running:
        cap = cv2.VideoCapture(0)  # Start the webcam feed
        if cap.isOpened():
            fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Attempt to get the FPS; default to 30 if it fails
        else:
            fps = 30  # Default value in case of failure
        running = True
        update_frame()

def stop_detection():
    """Function to stop the object detection process."""
    global cap, running
    running = False
    if cap:
        cap.release()
    cv2.destroyAllWindows()

def reset_alerts():
    """Function to reset all alert notifications."""
    global alert_person_ids
    alert_person_ids.clear()
    update_static_info_display()

def save_screenshot(frame, csv_path, row_num):
    """Function to save the screenshot to the Excel file."""
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp_file:
        screenshot_path = tmp_file.name
        cv2.imwrite(screenshot_path, frame)

    if csv_path:
        workbook = openpyxl.load_workbook(csv_path)
        sheet = workbook.active
        img = ExcelImage(screenshot_path)
        img.anchor = f'F{row_num}'  # Place image in column F at the correct row
        sheet.add_image(img)
        workbook.save(csv_path)
        os.remove(screenshot_path)


def update_frame():
    """Function to read a frame from the webcam, process it, and update static variables if changes are detected."""
    global cap, panel, counter, fps, elapsed, start_time
    global unique_track_ids, track_labels, track_times, track_positions, obstruction_frames, obstruction_blocked
    global alert_person_ids, current_person_count, current_camera_view_status, current_running_status
    global running_counters  # Add this line

    alert_person_ids.clear()

    if running and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            og_frame = frame.copy()
            results = model(frame, device=0, classes=0, conf=0.75)
            active_track_ids = set()

            for result in results:
                boxes = result.boxes
                cls = boxes.cls.tolist()
                xyxy = boxes.xyxy
                conf = boxes.conf
                xywh = boxes.xywh

            pred_cls = np.array(cls)
            conf = conf.detach().cpu().numpy()
            xyxy = xyxy.detach().cpu().numpy()
            bboxes_xywh = xywh.cpu().numpy()

            tracks = tracker.update(bboxes_xywh, conf, og_frame)

            # Reset running status
            new_running_status = "No Running"

            for track in tracker.tracker.tracks:
                track_id = track.track_id
                x1, y1, x2, y2 = track.to_tlbr()
                w = x2 - x1
                h = y2 - y1

                # Define color for bounding box
                red_color = (0, 0, 255)
                blue_color = (255, 0, 0)
                green_color = (0, 255, 0)
                color_id = track_id % 3
                color = red_color if color_id == 0 else blue_color if color_id == 1 else green_color
                cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

                if track_id not in track_labels:
                    track_labels[track_id] = "Person"
                    track_times[track_id] = 0
                    track_positions[track_id] = (x1, y1)
                    running_counters[track_id] = 0  # Initialize running counter

                track_times[track_id] += 1
                prev_x1, prev_y1 = track_positions[track_id]
                displacement = np.sqrt((x1 - prev_x1) ** 2 + (y1 - prev_y1) ** 2)

                # Check to avoid division by zero
                if fps > 0:
                    speed = displacement / fps
                else:
                    speed = 0

                track_positions[track_id] = (x1, y1)

                # Determine if the person is running
                if speed > running_threshold and w * h > 5000:
                    running_counters[track_id] += 1  # Increment counter for consecutive running frames
                    if running_counters[track_id] > fps/2:  # More than 1 second of running
                        track_labels[track_id] = "Running"
                        new_running_status = "Running Detected"
                else:
                    running_counters[track_id] = 0  # Reset counter if no running detected
                    track_labels[track_id] = "Person"

                total_seconds = track_times[track_id] / fps if fps > 0 else 0
                minutes = int(total_seconds // 60)
                seconds = int(total_seconds % 60)

                 # Trigger alert only after 60 seconds have passed
                if total_seconds > 60 and track_id not in alert_person_ids:
                    alert_person_ids.append(track_id)  # Append only unique IDs
                    update_static_info_display()

                cv2.putText(og_frame, f"{track_labels[track_id]} {minutes:02}:{seconds:02}", 
                           (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                active_track_ids.add(track_id)

            unique_track_ids.intersection_update(active_track_ids)
            unique_track_ids.update(active_track_ids)

            # Update static variables if changes are detected
            person_count = len(unique_track_ids)
            if person_count != current_person_count:
                current_person_count = person_count
                update_static_info_display()

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
            if blur < 50:
                obstruction_frames += 1
                if obstruction_frames > fps * 2:
                    obstruction_blocked = True
            else:
                obstruction_frames = 0
                obstruction_blocked = False

            # Update camera view status
            new_camera_view_status = "Blocked" if obstruction_blocked else "Normal"
            if new_camera_view_status != current_camera_view_status:
                current_camera_view_status = new_camera_view_status
                update_static_info_display()

            # Update running status
            if new_running_status != current_running_status:
                current_running_status = new_running_status
                update_static_info_display()

            current_time = time.perf_counter()
            elapsed = (current_time - start_time)
            counter += 1
            if elapsed > 1:
                fps = counter / elapsed
                counter = 0
                start_time = current_time

            # Convert the image to RGB (Tkinter-compatible format) and display it
            img = cv2.cvtColor(og_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)

            if panel is None:
                panel = Label(window)
                panel.pack(padx=10, pady=10)
            panel.imgtk = imgtk  # Keep a reference to avoid garbage collection
            panel.config(image=imgtk)  # Update the image in the panel
            window.after(10, update_frame)  # Schedule the next frame update


def update_static_info_display():
    """Function to display static variables without real-time changes."""
    global info_text, current_person_count, current_camera_view_status, current_running_status, alert_person_ids
    
    # Enable editing in the text widget
    info_text.config(state=tk.NORMAL)
    
    # Clear the current text
    info_text.delete(1.0, tk.END)

    # Change color to red if the person count exceeds 2
    if current_person_count > 2:
        info_text.tag_configure("count_exceed", foreground="red")
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n", "count_exceed")
    else:
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n")

    # Change color to red if the camera view is blocked
    if current_camera_view_status == "Blocked":
        info_text.tag_configure("blocked", foreground="red")
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n", "blocked")
    else:
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n")

    info_text.insert(tk.END, f"Running Status: {current_running_status}\n")

    # Display alert for prolonged stay
    if alert_person_ids:
        info_text.insert(tk.END, f"Prolonged Stay - Person IDs: {', '.join(map(str, alert_person_ids))} over 1 minute\n")
    else:
        info_text.insert(tk.END, "Prolonged Stay: NIL\n")

    # Disable editing in the text widget
    info_text.config(state=tk.DISABLED)
    
    # Automatically scroll to the end
    info_text.see(tk.END)



def on_close():
    """Function to handle window close event."""
    stop_detection()
    window.destroy()


# UI Controls and Layout
button_frame = Frame(window)
button_frame.pack(side="top", pady=10)

start_button = Button(button_frame, text="Start Detection", command=start_detection)
start_button.pack(side="left", padx=5)

stop_button = Button(button_frame, text="Stop Detection", command=stop_detection)
stop_button.pack(side="left", padx=5)

reset_button = Button(button_frame, text="Reset Alerts", command=reset_alerts)
reset_button.pack(side="left", padx=5)

# Add a frame for displaying inference overlays
info_frame = Frame(window)
info_frame.pack(side="bottom", fill="x")

scrollbar = Scrollbar(info_frame, orient=VERTICAL)
scrollbar.pack(side=RIGHT, fill=Y)

info_text = Text(info_frame, height=8, wrap=tk.WORD, yscrollcommand=scrollbar.set)
info_text.pack(side="left", fill="x", padx=5, pady=5)
scrollbar.config(command=info_text.yview)

window.protocol("WM_DELETE_WINDOW", on_close)
window.mainloop()

In [ ]:
#Weapon Detection

In [16]:


# Initialize YOLO model and DeepSort tracker
model = YOLO("best.pt")


class_names = ["person", "weapon"]  # Ensure this matches your model's class names

# Tkinter window setup
window = tk.Tk()
window.title("Real-time Object Detection")
window.geometry("1920x1080")

# Global variables
cap = None
video_path = None  # Path for video file
csv_path = None  # Path for CSV file
running = False  # To control the webcam feed loop
panel = None  # Panel for video display
info_text = None  # Text widget to display inference information

# Static variables to track changes
current_person_count = 0
current_camera_view_status = "Normal"
current_running_status = "No Running Detected"
current_weapon_status = "No Weapon Detected"  # New variable for weapon detection
alert_person_ids = []

# Variables for tracking
counter, fps, elapsed = 0, 1, 0  # Initialize fps to 1 to avoid division by zero
start_time = time.perf_counter()
unique_track_ids = set()
track_labels = {}
track_times = {}
track_positions = {}
obstruction_frames = 0
obstruction_blocked = False
running_threshold = 0.5
obstruction_threshold = 0.60
running_counters = {}  # Dictionary to track consecutive frames of running detection

def start_detection():
    """Function to start the object detection process."""
    global cap, running, fps, video_path
    if not running:
        if video_path:
            cap = cv2.VideoCapture(video_path)  # Use video file if path is provided
        else:
            cap = cv2.VideoCapture(0)  # Start the webcam feed
        if cap.isOpened():
            fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Attempt to get the FPS; default to 30 if it fails
        else:
            fps = 30  # Default value in case of failure
        running = True
        update_frame()

def stop_detection():
    """Function to stop the object detection process."""
    global cap, running
    running = False
    if cap:
        cap.release()
    cv2.destroyAllWindows()

def reset_alerts():
    """Function to reset all alert notifications."""
    global alert_person_ids
    alert_person_ids.clear()
    update_static_info_display()

def save_to_csv():
    """Function to save data to CSV."""
    global csv_path
    if csv_path:
        data = {
            "Time": [time.strftime("%Y-%m-%d %H:%M:%S")],
            "Person Count": [current_person_count],
            "Running Status": [current_running_status],
            "Weapon Status": [current_weapon_status],
            "Camera View Status": [current_camera_view_status]
        }
        df = pd.DataFrame(data)
        df.to_csv(csv_path, mode='a', header=False, index=False)
        print(f"Data saved to {csv_path}")

def update_frame():
    """Function to read a frame from the webcam or video file, process it, and update static variables if changes are detected."""
    global cap, panel, counter, fps, elapsed, start_time
    global     unique_track_ids, track_labels, track_times, track_positions, obstruction_frames, obstruction_blocked
    global alert_person_ids, current_person_count, current_camera_view_status, current_running_status, current_weapon_status
    global running_counters

    alert_person_ids.clear()

    if running and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            og_frame = frame.copy()
            results = model(frame, device=0, conf=0.70)
            active_track_ids = set()

            # Example mapping: update based on your model's class indices
            class_map = {0: "Person", 1: "Weapon"}  # Adjust indices based on model output

            for result in results:
                boxes = result.boxes
                cls = boxes.cls.tolist()
                xyxy = boxes.xyxy
                conf = boxes.conf
                xywh = boxes.xywh

            # Map class indices to labels with error handling
            try:
                pred_cls = [class_map[int(c)] for c in cls]  # Ensure class labels are correctly mapped
            except KeyError as e:
                print(f"Warning: Class index {e} not found in class_map")
                pred_cls = ["Unknown"] * len(cls)  # Assign "Unknown" if class index not found

            # Ensure arrays are properly handled
            conf = conf.detach().cpu().numpy()
            xyxy = xyxy.detach().cpu().numpy()
            bboxes_xywh = xywh.cpu().numpy()
            tracks = tracker.update(bboxes_xywh, conf, og_frame)

            # Reset running and weapon status
            new_running_status = "No Running Detected"
            new_weapon_status = "No Weapon Detected"

            # Process each track
            for track in tracker.tracker.tracks:
                track_id = track.track_id
                x1, y1, x2, y2 = track.to_tlbr()
                w = x2 - x1
                h = y2 - y1

                # Define color for bounding box based on track_id
                colors = [(0, 0, 255), (255, 0, 0), (0, 255, 0)]  # Red, Blue, Green
                color = colors[track_id % len(colors)]
                cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

                # Initialize tracking info if not already present
                if track_id not in track_labels:
                    track_labels[track_id] = "Person"
                    track_times[track_id] = 0
                    track_positions[track_id] = (x1, y1)
                    running_counters[track_id] = 0

                track_times[track_id] += 1
                prev_x1, prev_y1 = track_positions[track_id]
                displacement = np.sqrt((x1 - prev_x1) ** 2 + (y1 - prev_y1) ** 2)

                # Calculate speed
                speed = displacement / fps if fps > 0 else 0
                track_positions[track_id] = (x1, y1)

                # Determine if the person is running or has a weapon
                if "Weapon" in pred_cls:
                    track_labels[track_id] = "Weapon"
                    new_weapon_status = "Weapon Detected"
                elif speed > running_threshold and w * h > 5000:
                    running_counters[track_id] += 1  # Increment counter for consecutive running frames
                    if running_counters[track_id] > fps / 2:  # More than 1 second of running
                        track_labels[track_id] = "Running"
                        new_running_status = "Running Detected"
                else:
                    running_counters[track_id] = 0  # Reset counter if no running detected
                    track_labels[track_id] = "Person"  # Only set to "Person" if no other conditions met

                total_seconds = track_times[track_id] / fps if fps > 0 else 0
                minutes = int(total_seconds // 60)
                seconds = int(total_seconds % 60)

                # Trigger alert only after 60 seconds have passed
                if total_seconds > 60 and track_id not in alert_person_ids:
                    alert_person_ids.append(track_id)  # Append only unique IDs

                cv2.putText(og_frame, f"{track_labels[track_id]} {minutes:02}:{seconds:02}", 
                           (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                active_track_ids.add(track_id)

            unique_track_ids.intersection_update(active_track_ids)
            unique_track_ids.update(active_track_ids)

            # Update static variables if changes are detected
            person_count = len(unique_track_ids)
            if person_count != current_person_count:
                current_person_count = person_count
                save_to_csv()  # Save to CSV when person count changes

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
            if blur < 50:
                obstruction_frames += 1
                obstruction_blocked = obstruction_frames > fps * 2
            else:
                obstruction_frames = 0
                obstruction_blocked = False

            # Update camera view status
            new_camera_view_status = "Blocked" if obstruction_blocked else "Normal"
            if new_camera_view_status != current_camera_view_status:
                current_camera_view_status = new_camera_view_status
                save_to_csv()  # Save to CSV when camera view status changes

            if (new_running_status != current_running_status or 
                new_weapon_status != current_weapon_status or 
                new_camera_view_status != current_camera_view_status):
                current_running_status = new_running_status
                current_weapon_status = new_weapon_status
                update_static_info_display()

            current_time = time.perf_counter()
            elapsed = (current_time - start_time)
            counter += 1
            if elapsed > 1:
                fps = counter / elapsed
                counter = 0
                start_time = current_time

            # Convert the image to RGB (Tkinter-compatible format) and display it
            img = cv2.cvtColor(og_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)

            if panel is None:
                panel = Label(window)
                panel.pack(padx=10, pady=10)
            panel.imgtk = imgtk  # Keep a reference to avoid garbage collection
            panel.config(image=imgtk)  # Update the image in the panel
            window.after(10, update_frame)  # Schedule the next frame update

def update_static_info_display():
    """Function to display static variables without real-time changes."""
    global info_text, current_person_count, current_camera_view_status, current_running_status, current_weapon_status, alert_person_ids
    
    # Enable editing in the text widget
    info_text.config(state=tk.NORMAL)
    
    # Clear the current text
    info_text.delete(1.0, tk.END)

    # Change color to red if the person count exceeds 2
    if current_person_count > 2:
        info_text.tag_configure("count_exceed", foreground="red")
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n", "count_exceed")
    else:
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n")

    # Change color to red if the camera view is blocked
    if current_camera_view_status == "Blocked":
        info_text.tag_configure("camera_blocked", foreground="red")
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n", "camera_blocked")
    else:
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n")

    # Change color to red if running is detected
    if current_running_status == "Running Detected":
        info_text.tag_configure("running_detected", foreground="red")
        info_text.insert(tk.END, f"Running Status: {current_running_status}\n", "running_detected")
    else:
        info_text.insert(tk.END, f"Running Status: {current_running_status}\n")

    # Change color to red if a weapon is detected
    if current_weapon_status == "Weapon Detected":
        info_text.tag_configure("weapon_detected", foreground="red")
        info_text.insert(tk.END, f"Weapon Status: {current_weapon_status}\n", "weapon_detected")
    else:
        info_text.insert(tk.END, f"Weapon Status: {current_weapon_status}\n")

    # Display alert for prolonged stay
    if alert_person_ids:
        info_text.insert(tk.END, f"Prolonged Stay - Person IDs: {', '.join(map(str, alert_person_ids))} over 1 minute\n")
    else:
        info_text.insert(tk.END, "Prolonged Stay: NIL\n")

    # Disable editing in the text widget
    info_text.config(state=tk.DISABLED)
    # Automatically scroll to the end
    info_text.see(tk.END)

def reset_csv_path():
    """Function to assign a new CSV path."""
    global csv_path
    csv_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    print(f"CSV path set to {csv_path}")

def upload_video():
    """Function to upload a video file."""
    global video_path
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi")])
    print(f"Video path set to {video_path}")

def on_close():
    """Function to handle window close event."""
    stop_detection()
    window.destroy()

# UI Controls and Layout
button_frame = Frame(window)
button_frame.pack(side="top", pady=10)

start_button = Button(button_frame, text="Start Detection", command=start_detection)
start_button.pack(side="left", padx=5)

stop_button = Button(button_frame, text="Stop Detection", command=stop_detection)
stop_button.pack(side="left", padx=5)

reset_button = Button(button_frame, text="Reset Alerts", command=reset_alerts)
reset_button.pack(side="left", padx=5)

csv_button = Button(button_frame, text="Set CSV Path", command=reset_csv_path)
csv_button.pack(side="left", padx=5)

video_button = Button(button_frame, text="Upload Video", command=upload_video)
video_button.pack(side="left", padx=5)

# Add a frame for displaying inference overlays
info_frame = Frame(window)
info_frame.pack(side="bottom", fill="x")

scrollbar = Scrollbar(info_frame, orient=VERTICAL)
scrollbar.pack(side=RIGHT, fill=Y)

info_text = Text(info_frame, height=8, wrap=tk.WORD, yscrollcommand=scrollbar.set)
info_text.pack(side="left", fill="x", padx=5, pady=5)
scrollbar.config(command=info_text.yview)

window.protocol("WM_DELETE_WINDOW", on_close)
window.mainloop()




0: 480x640 2 persons, 33.2ms
Speed: 6.0ms preprocess, 33.2ms inference, 10.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 18.5ms
Speed: 2.8ms preprocess, 18.5ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 12.4ms
Speed: 2.0ms preprocess, 12.4ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 11.0ms
Speed: 2.5ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 12.5ms
Speed: 2.0ms preprocess, 12.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 13.1ms
Speed: 2.0ms preprocess, 13.1ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 13.9ms
Speed: 2.0ms preprocess, 13.9ms inference, 2.0ms postprocess per image at shape 

In [11]:
# Initialize YOLO model and DeepSort tracker
model = YOLO("best.pt")


class_names = ["person", "weapon"]  # Ensure this matches your model's class names

# Tkinter window setup
window = tk.Tk()
window.title("SAD - SUSPICIOUS ACTIVITY DETECTOR")
window.geometry("1920x1080")

# Global variables
cap = None
video_path = None  # Path for video file
csv_path = None  # Path for CSV file
running = False  # To control the webcam feed loop
panel = None  # Panel for video display
info_text = None  # Text widget to display inference information

# Static variables to track changes
current_person_count = 0
current_camera_view_status = "Normal"
current_running_status = "No Running Detected"
current_weapon_status = "No Weapon Detected"  # New variable for weapon detection
alert_person_ids = []

# Variables for tracking
counter, fps, elapsed = 0, 1, 0  # Initialize fps to 1 to avoid division by zero
start_time = time.perf_counter()
unique_track_ids = set()
track_labels = {}
track_times = {}
track_positions = {}
obstruction_frames = 0
obstruction_blocked = False
running_threshold = 0.5
obstruction_threshold = 0.60
running_counters = {}  # Dictionary to track consecutive frames of running detection

def start_detection():
    """Function to start the object detection process."""
    global cap, running, fps, video_path
    if not running:
        if video_path:
            cap = cv2.VideoCapture(video_path)  # Use video file if path is provided
        else:
            cap = cv2.VideoCapture(0)  # Start the webcam feed
        if cap.isOpened():
            fps = cap.get(cv2.CAP_PROP_FPS) or 30  # Attempt to get the FPS; default to 30 if it fails
        else:
            fps = 30  # Default value in case of failure
        running = True
        update_frame()

def stop_detection():
    """Function to stop the object detection process."""
    global cap, running
    running = False
    if cap:
        cap.release()
    cv2.destroyAllWindows()

def reset_alerts():
    """Function to reset all alert notifications."""
    global alert_person_ids
    alert_person_ids.clear()
    update_static_info_display()
    
    
def save_screenshot(frame, csv_path, row_num):
    """Function to save the screenshot to the Excel file."""
    try:
        with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp_file:
            screenshot_path = tmp_file.name
            cv2.imwrite(screenshot_path, frame)
            print(f"Screenshot saved temporarily at {screenshot_path}")

        if csv_path:
            workbook = openpyxl.load_workbook(csv_path)
            sheet = workbook.active
            img = openpyxl.drawing.image.Image(screenshot_path)
            img.anchor = f'F{row_num}'  # Place image in column F at the correct row
            sheet.add_image(img)
            workbook.save(csv_path)
            print(f"Image inserted into Excel at F{row_num}")
            os.remove(screenshot_path)  # Clean up the temporary image file
    except Exception as e:
        print(f"Error saving screenshot to Excel: {e}")




def save_to_xlsx(csv_path, time, current_person_count, current_running_status, current_weapon_status, current_camera_view_status, screenshot_frame=None):
    """Function to save data to XLSX with enhanced formatting for a professional report."""
    # Define border style
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )

    if csv_path:
        # Load the workbook if it exists, otherwise create a new one
        try:
            workbook = openpyxl.load_workbook(csv_path)
            sheet = workbook.active
        except FileNotFoundError:
            workbook = Workbook()
            sheet = workbook.active
            # Add column headers with bold formatting
            headers = ["Time Frame", "Person Count", "Running Status", "Weapon Status", "Camera View Status", "Screenshot Path"]
            for col_num, header in enumerate(headers, 1):
                cell = sheet.cell(row=1, column=col_num)
                cell.value = header
                cell.font = Font(bold=True)  # Make the header bold
                cell.alignment = Alignment(horizontal='center', vertical='center')  # Center align headers
                cell.border = thin_border  # Add border to headers

        # Prepare data row
        screenshot_path = None
        row_num = sheet.max_row + 1
        if screenshot_frame is not None:
            screenshot_path = save_screenshot(screenshot_frame, csv_path, row_num)
        
        data = [
            time.strftime("%Y-%m-%d %H:%M:%S"),
            current_person_count,
            current_running_status,
            current_weapon_status,
            current_camera_view_status,
            screenshot_path  # Add screenshot path to data
        ]
        
        for col_num, value in enumerate(data, 1):
            cell = sheet.cell(row=row_num, column=col_num)
            cell.value = value
            cell.alignment = Alignment(horizontal='center', vertical='center')  # Center align cells
            cell.border = thin_border  # Add border to data cells

        # Adjust the column width for better spacing
        column_widths = [20, 15, 20, 20, 25, 30]
        for i, width in enumerate(column_widths, 1):
            sheet.column_dimensions[openpyxl.utils.get_column_letter(i)].width = width

        # Save the workbook
        workbook.save(csv_path)
        print(f"Data saved to {csv_path}")

        
        
def update_frame():
    """Function to read a frame from the webcam or video file, process it, and update static variables if changes are detected."""
    global cap, panel, counter, fps, elapsed, start_time
    global     unique_track_ids, track_labels, track_times, track_positions, obstruction_frames, obstruction_blocked
    global alert_person_ids, current_person_count, current_camera_view_status, current_running_status, current_weapon_status
    global running_counters

    alert_person_ids.clear()

    if running and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            og_frame = frame.copy()
            results = model(frame, device=0, conf=0.70)
            active_track_ids = set()

            # Example mapping: update based on your model's class indices
            class_map = {0: "Person", 1: "Weapon"}  # Adjust indices based on model output

            for result in results:
                boxes = result.boxes
                cls = boxes.cls.tolist()
                xyxy = boxes.xyxy
                conf = boxes.conf
                xywh = boxes.xywh

            # Map class indices to labels with error handling
            try:
                pred_cls = [class_map[int(c)] for c in cls]  # Ensure class labels are correctly mapped
            except KeyError as e:
                print(f"Warning: Class index {e} not found in class_map")
                pred_cls = ["Unknown"] * len(cls)  # Assign "Unknown" if class index not found

            # Ensure arrays are properly handled
            conf = conf.detach().cpu().numpy()
            xyxy = xyxy.detach().cpu().numpy()
            bboxes_xywh = xywh.cpu().numpy()
            tracks = tracker.update(bboxes_xywh, conf, og_frame)

            # Reset running and weapon status
            new_running_status = "No Running Detected"
            new_weapon_status = "No Weapon Detected"

            # Process each track
            for track in tracker.tracker.tracks:
                track_id = track.track_id
                x1, y1, x2, y2 = track.to_tlbr()
                w = x2 - x1
                h = y2 - y1

                # Define color for bounding box based on track_id
                colors = [(0, 0, 255), (255, 0, 0), (0, 255, 0)]  # Red, Blue, Green
                color = colors[track_id % len(colors)]
                cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

                # Initialize tracking info if not already present
                if track_id not in track_labels:
                    track_labels[track_id] = "Person"
                    track_times[track_id] = 0
                    track_positions[track_id] = (x1, y1)
                    running_counters[track_id] = 0

                track_times[track_id] += 1
                prev_x1, prev_y1 = track_positions[track_id]
                displacement = np.sqrt((x1 - prev_x1) ** 2 + (y1 - prev_y1) ** 2)

                # Calculate speed
                speed = displacement / fps if fps > 0 else 0
                track_positions[track_id] = (x1, y1)

                # Determine if the person is running or has a weapon
                if "Weapon" in pred_cls:
                    track_labels[track_id] = "Weapon"
                    new_weapon_status = "Weapon Detected"
                    save_to_xlsx(csv_path, time, current_person_count, current_running_status, current_weapon_status, current_camera_view_status, screenshot_frame=og_frame)
                elif speed > running_threshold and w * h > 5000:
                    running_counters[track_id] += 1  # Increment counter for consecutive running frames
                    if running_counters[track_id] > fps / 2:  # More than 1 second of running
                        track_labels[track_id] = "Running"
                        new_running_status = "Running Detected"
                else:
                    running_counters[track_id] = 0  # Reset counter if no running detected
                    track_labels[track_id] = "Person"  # Only set to "Person" if no other conditions met
                update_static_info_display()

                total_seconds = track_times[track_id] / fps if fps > 0 else 0
                minutes = int(total_seconds // 60)
                seconds = int(total_seconds % 60)

                # Trigger alert only after 60 seconds have passed
                if total_seconds > 60 and track_id not in alert_person_ids:
                    alert_person_ids.append(track_id)  # Append only unique IDs
                    update_static_info_display()

                cv2.putText(og_frame, f"{track_labels[track_id]} {minutes:02}:{seconds:02}", 
                           (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                active_track_ids.add(track_id)

            unique_track_ids.intersection_update(active_track_ids)
            unique_track_ids.update(active_track_ids)

            # Update static variables if changes are detected
            person_count = len(unique_track_ids)
            if person_count != current_person_count:
                current_person_count = person_count
                save_to_xlsx(csv_path, time, current_person_count, current_running_status, current_weapon_status, current_camera_view_status)


            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
            if blur < 50:
                obstruction_frames += 1
                obstruction_blocked = obstruction_frames > fps * 1
            else:
                obstruction_frames = 0
                obstruction_blocked = False

            # Update camera view status
            new_camera_view_status = "Blocked" if obstruction_blocked else "Normal"
            update_static_info_display()

            if new_camera_view_status != current_camera_view_status:
                current_camera_view_status = new_camera_view_status
                save_to_xlsx(csv_path, time, current_person_count, current_running_status, current_weapon_status, current_camera_view_status)


            if (new_running_status != current_running_status or 
                new_weapon_status != current_weapon_status or 
                new_camera_view_status != current_camera_view_status):
                current_running_status = new_running_status
                current_weapon_status = new_weapon_status
                update_static_info_display()

            current_time = time.perf_counter()
            elapsed = (current_time - start_time)
            counter += 1
            if elapsed > 1:
                fps = counter / elapsed
                counter = 0
                start_time = current_time

            # Convert the image to RGB (Tkinter-compatible format) and display it
            img = cv2.cvtColor(og_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)

            if panel is None:
                panel = Label(window)
                panel.pack(padx=10, pady=10)
            panel.imgtk = imgtk  # Keep a reference to avoid garbage collection
            panel.config(image=imgtk)  # Update the image in the panel
            window.after(10, update_frame)  # Schedule the next frame update

def update_static_info_display():
    """Function to display static variables without real-time changes."""
    global info_text, current_person_count, current_camera_view_status, current_running_status, current_weapon_status, alert_person_ids
    
    # Enable editing in the text widget
    info_text.config(state=tk.NORMAL)
    
    # Clear the current text
    info_text.delete(1.0, tk.END)

    # Change color to red if the person count exceeds 2
    if current_person_count > 2:
        info_text.tag_configure("count_exceed", foreground="red")
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n", "count_exceed")
    else:
        info_text.insert(tk.END, f"Person Count: {current_person_count}\n")

    # Change color to red if the camera view is blocked
    if current_camera_view_status == "Blocked":
        info_text.tag_configure("camera_blocked", foreground="red")
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n", "camera_blocked")
    else:
        info_text.insert(tk.END, f"Camera View: {current_camera_view_status}\n")

    # Change color to red if running is detected
    if current_running_status == "Running Detected":
        info_text.tag_configure("running_detected", foreground="red")
        info_text.insert(tk.END, f"Running Status: {current_running_status}\n", "running_detected")
    else:
        info_text.insert(tk.END, f"Running Status: {current_running_status}\n")

    # Change color to red if a weapon is detected
    if current_weapon_status == "Weapon Detected":
        info_text.tag_configure("weapon_detected", foreground="red")
        info_text.insert(tk.END, f"Weapon Status: {current_weapon_status}\n", "weapon_detected")
    else:
        info_text.insert(tk.END, f"Weapon Status: {current_weapon_status}\n")

    # Display alert for prolonged stay
    if alert_person_ids:
        info_text.insert(tk.END, f"Prolonged Stay - Person IDs: {', '.join(map(str, alert_person_ids))} over 1 minute\n")
    else:
        info_text.insert(tk.END, "Prolonged Stay: NIL\n")

    # Disable editing in the text widget
    info_text.config(state=tk.DISABLED)
    # Automatically scroll to the end
    info_text.see(tk.END)

def reset_csv_path():
    """Function to assign a new CSV path."""
    global csv_path
    csv_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("xlsx files", "*.xlsx")])
    print(f"CSV path set to {csv_path}")

def upload_video():
    """Function to upload a video file."""
    global video_path
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi")])
    print(f"Video path set to {video_path}")

def on_close():
    """Function to handle window close event."""
    stop_detection()
    window.destroy()

# UI Controls and Layout
button_frame = Frame(window)
button_frame.pack(side="top", pady=10)

start_button = Button(button_frame, text="Start Detection", command=start_detection)
start_button.pack(side="left", padx=5)

stop_button = Button(button_frame, text="Stop Detection", command=stop_detection)
stop_button.pack(side="left", padx=5)

reset_button = Button(button_frame, text="Reset Alerts", command=reset_alerts)
reset_button.pack(side="left", padx=5)

csv_button = Button(button_frame, text="Set XLSX Path", command=reset_csv_path)
csv_button.pack(side="left", padx=5)

video_button = Button(button_frame, text="Upload Video", command=upload_video)
video_button.pack(side="left", padx=5)

# Add a frame for displaying inference overlays
info_frame = Frame(window)
info_frame.pack(side="bottom", fill="x")

scrollbar = Scrollbar(info_frame, orient=VERTICAL)
scrollbar.pack(side=RIGHT, fill=Y)

info_text = Text(info_frame, height=8, wrap=tk.WORD, yscrollcommand=scrollbar.set)
info_text.pack(side="left", fill="x", padx=5, pady=5)
scrollbar.config(command=info_text.yview)

window.protocol("WM_DELETE_WINDOW", on_close)
window.mainloop()

CSV path set to C:/Users/trvig/OneDrive/Desktop/weapon detection/test2.xlsx
Video path set to C:/Users/trvig/OneDrive/Desktop/weapon detection/test.mp4

0: 384x640 1 person, 1 weapon, 39.9ms
Speed: 5.3ms preprocess, 39.9ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Screenshot saved temporarily at C:\Users\trvig\AppData\Local\Temp\tmpbvaj_pjd.png
Image inserted into Excel at F2
Data saved to C:/Users/trvig/OneDrive/Desktop/weapon detection/test2.xlsx
Screenshot saved temporarily at C:\Users\trvig\AppData\Local\Temp\tmprj3x_j1p.png
Image inserted into Excel at F3
Data saved to C:/Users/trvig/OneDrive/Desktop/weapon detection/test2.xlsx
Data saved to C:/Users/trvig/OneDrive/Desktop/weapon detection/test2.xlsx

0: 384x640 1 person, 1 weapon, 16.0ms
Speed: 3.2ms preprocess, 16.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
Screenshot saved temporarily at C:\Users\trvig\AppData\Local\Temp\tmpiucb6bnk.png
Image inserted into Excel at F5
Data saved t